In [ ]:
import json

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

from flask import Flask
from flask import request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/linreg")
def lin_reg():
    
    args = request.args
    
    X = json.loads(args['X']) 
    y = json.loads(args['y'])
    fit_intercept = json.loads(args['fit_intercept'])

    X = np.array(X).astype(float)
    y = np.array(y).astype(float)[0]
    fit_intercept = bool(fit_intercept)
    
    LR = LinearRegression(fit_intercept = fit_intercept)
    reg = LR.fit(X, y)
    
    out = {
        'score': reg.score(X, y),
        'coefs': reg.coef_.tolist(),
        'intercept': reg.intercept_,
    }
    
    return out

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

@app.route("/sentiment")
def sentiment():
    args = request.args
    
    a = args['a']
    return sid.polarity_scores(a)
    

import base64
import seaborn as sns
import matplotlib.pyplot as plt
from flask import render_template, Response, send_file
import io
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

@app.route("/heatmap")
def heat_map():
    args = request.args
    
    X = json.loads(args['X']) 
    y = json.loads(args['y'])
    
    X = np.array(X).astype(float)
    y = np.array(y).astype(float)[0]
    
    y = np.array([y])
    uniform_data = np.concatenate((X, y.T), axis=1)
    corr = np.corrcoef(uniform_data)
    img = sns.heatmap(corr)

    my_stringIObytes = io.BytesIO()
    my_stringIObytes.seek(0)
    FigureCanvas(img.figure).print_png(my_stringIObytes)
    return Response(my_stringIObytes.getvalue(), mimetype='image/png')


@app.route("/heatmap2")
def heat_map2():
    args = request.args

    X = json.loads(args['X']) 
    y = json.loads(args['y'])

    X = np.array(X).astype(float)
    y = np.array(y).astype(float)[0]

    y = np.array([y])
    uniform_data = np.concatenate((X, y.T), axis=1)
    corr = np.corrcoef(uniform_data)
    img = sns.heatmap(corr, cmap='coolwarm', cbar=False)
    fig = img.figure
    return nocache(fig_response(fig))

def fig_response(fig):
    """Turn a matplotlib Figure into Flask response"""
    img_bytes = io.BytesIO()
    fig.savefig(img_bytes)
    img_bytes.seek(0)
    return send_file(img_bytes, mimetype='image/png')

def nocache(response):
    """Add Cache-Control headers to disable caching a response"""
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, max-age=0'
    return response

app.run()